In [3]:
!brew install libsndfile

==> Downloading https://formulae.brew.sh/api/formula.jws.json
==> Downloading https://formulae.brew.sh/api/cask.jws.json
To reinstall 1.2.2_1, run:
  brew reinstall libsndfile


In [11]:
!pip install torchaudio

  Using cached torchaudio-2.6.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.6 kB)
Using cached torchaudio-2.6.0-cp312-cp312-macosx_11_0_arm64.whl (1.8 MB)


In [13]:
pip install transformers torch speechbrain torchaudio langdetect warnings

ERROR: Could not find a version that satisfies the requirement warnings (from versions: none)
ERROR: No matching distribution found for warnings
Note: you may need to restart the kernel to use updated packages.


In [15]:
import torch
import torchaudio
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
from speechbrain.pretrained import EncoderDecoderASR
from langdetect import detect, LangDetectException
import warnings

In [27]:
print("Loading translation model (MBART)...")
# Load MBART model and tokenizer for translation
model_name = "facebook/mbart-large-50-many-to-many-mmt"
try:
    tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
    model = MBartForConditionalGeneration.from_pretrained(model_name)
    print("Translation model loaded successfully.")
except Exception as e:
    print(f"Error loading translation model: {e}")
    exit() # Exit if essential models fail to load

Loading translation model (MBART)...
Translation model loaded successfully.


In [29]:
print("Loading speech-to-text model (SpeechBrain)...")
# Load speech-to-text model
try:
    asr_model = EncoderDecoderASR.from_hparams(
        source="speechbrain/asr-crdnn-rnnlm-librispeech", savedir="tmpdir_asr"
    )
    print("Speech-to-text model loaded successfully.")
except Exception as e:
    print(f"Error loading speech-to-text model: {e}")
    # Decide if you want to exit or continue without speech functionality
    # exit() 
    asr_model = None # Allow continuing without speech if model load fails
    print("Warning: Could not load ASR model. Speech input will not work.")

INFO:speechbrain.utils.fetching:Fetch hyperparams.yaml: Fetching from HuggingFace Hub 'speechbrain/asr-crdnn-rnnlm-librispeech' if not cached


Loading speech-to-text model (SpeechBrain)...


INFO:speechbrain.utils.fetching:Fetch custom.py: Fetching from HuggingFace Hub 'speechbrain/asr-crdnn-rnnlm-librispeech' if not cached
/opt/anaconda3/lib/python3.12/site-packages/speechbrain/utils/autocast.py:68: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  wrapped_fwd = torch.cuda.amp.custom_fwd(fwd, cast_inputs=cast_inputs)
INFO:speechbrain.utils.fetching:Fetch normalizer.ckpt: Fetching from HuggingFace Hub 'speechbrain/asr-crdnn-rnnlm-librispeech' if not cached
INFO:speechbrain.utils.fetching:Fetch asr.ckpt: Fetching from HuggingFace Hub 'speechbrain/asr-crdnn-rnnlm-librispeech' if not cached
INFO:speechbrain.utils.fetching:Fetch lm.ckpt: Fetching from HuggingFace Hub 'speechbrain/asr-crdnn-rnnlm-librispeech' if not cached
INFO:speechbrain.utils.fetching:Fetch tokenizer.ckpt: Fetching from HuggingFace Hub 'speechbrain/asr-crdnn-rnnlm-librispeech' if not cached
INFO:speechbrain.utils.param

Speech-to-text model loaded successfully.


In [33]:
# --- Step 2: Define Helper Functions ---
def speech_to_text(audio_path):
    """Transcribes speech from an audio file path."""
    if not asr_model:
        print("ASR model not available.")
        return None
    try:
        print(f"Transcribing audio file: {audio_path}")
        text = asr_model.transcribe_file(audio_path)
        print(f"Transcription result: {text}")
        return text
    except Exception as e:
        print(f"Error during speech transcription: {e}")
        return None

In [35]:
def detect_language(text):
    """Detects the language of the input text."""
    try:
        lang_code = detect(text)
        print(f"Detected language code: {lang_code}")
        return lang_code
    except LangDetectException:
        print("Language detection failed, defaulting to English.")
        return 'en' # Default to English if detection fails
    except Exception as e:
        print(f"An unexpected error occurred during language detection: {e}")
        return 'en' # Default to English on other errors

In [39]:
# --- Step 3: Language Code Mapping for MBART ---
lang_code_map = {
    'en': 'en_XX',   #english
    'fr': 'fr_XX',   #french
    'hi': 'hi_IN',   #hindi
    'gu': 'gu_IN',   #gujarati
    'ur': 'ur_PK',   #urdu
    'it': 'it_IT',   #italian
    'zh-cn': 'zh_CN', #Chinese
    'ja': 'ja_XX',   # Japanese
    'ar': 'ar_AR',   # Arabic
    'ko': 'ko_KR',   # Korean
    'ne': 'ne_NP',   # Nepali 
    'bn': 'bn_IN',   # Bengali
    'ml': 'ml_IN',   # malayalam
    'mr': 'mr_IN',   # marathi
    'ta': 'ta_IN',   # tamil
    'te': 'te_IN',   # telugu
    # Add more mappings as needed based on MBART's supported languages
}

In [41]:
def translate(text, src_lang_code, tgt_lang_code):
    """Translates text from source language to target language using MBART."""
    try:
        tokenizer.src_lang = src_lang_code
        encoded = tokenizer(text, return_tensors="pt")
        
        # Ensure the model and inputs are on the same device (CPU or GPU)
        # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        # model.to(device)
        # encoded = {k: v.to(device) for k, v in encoded.items()}

        generated_tokens = model.generate(
            **encoded,
            forced_bos_token_id=tokenizer.lang_code_to_id[tgt_lang_code]
        )
        translated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
        print(f"Translated '{text}' ({src_lang_code}) to '{translated_text}' ({tgt_lang_code})")
        return translated_text
    except KeyError:
        print(f"Error: MBART does not support the target language code '{tgt_lang_code}' derived from map.")
        # Fallback: return original text or an error message
        return text # Or return f"Error: Translation to {tgt_lang_code} not supported."
    except Exception as e:
        print(f"Error during translation: {e}")
        # Fallback: return original text or an error message
        return text # Or return "Error during translation."


In [43]:
# --- Step 4: Medicine Information ---
# Assume your medicine data is in a dictionary (keys should be lowercase)
medicine_data = {
    "paracetamol": "Used for fever and mild pain relief.",
    "ibuprofen": "Anti-inflammatory and pain-relieving medicine.",
    "aspirin": "Used to reduce pain, fever, or inflammation.",
    "loratadine": "Antihistamine used to treat allergies.",
    "amoxicillin": "Antibiotic used to treat bacterial infections.",
    # Add more medicines in lowercase
}

In [45]:
def get_medicine_info(med_name_key):
    """Retrieves medicine information from the dictionary (case-insensitive)."""
    # Assumes med_name_key is already lowercase and validated
    return medicine_data.get(med_name_key, "Medicine information not found.")

In [72]:
# --- Step 5: Main Chatbot Logic ---
def medicine_chatbot(input_text=None, audio_path=None):
    """Handles the chatbot interaction flow."""
    # === Part 1: Get Text Input ===
    if audio_path:
        print(f"\n--- Processing Audio Input: {audio_path} ---")
        input_text = speech_to_text(audio_path)
        if not input_text:
            return "Sorry, I couldn't understand the audio."
    elif input_text:
         print(f"\n--- Processing Text Input: {input_text} ---")
    else:
        return "Error: No input provided."
        
    # === Part 2: Detect Language ===
    detected_lang = detect_language(input_text)
    
    # === Part 3: Get MBART Source Language Code ===
    # Default to English if detected language is not in our map
    src_mbart_code = lang_code_map.get(detected_lang, 'en_XX')
    if detected_lang not in lang_code_map:
        print(f"Warning: Detected language '{detected_lang}' not in lang_code_map. Assuming English ('en_XX') for translation source.")
        # Optional: you could return an error here if you don't want to default
        # return f"Sorry, language '{detected_lang}' is not currently supported."

    # === Part 4: Translate Input to English (for lookup) ===
    # Only translate if the detected language isn't English
    if src_mbart_code != 'en_XX':
        eng_text = translate(input_text, src_mbart_code, 'en_XX')
    else:
        eng_text = input_text # Already in English (or defaulted to it)
    
    if not eng_text: # Handle potential translation failure
         return "Sorry, there was an error processing your request."

    # === Part 5: Extract Medicine Name & Get Information ===
    found_med_name = None
    med_info_eng = "Could not identify a known medicine in your query." # Default response

    for med_name_key in medicine_data.keys():
        # Simple substring check (lowercase) - might need refinement for accuracy
        if med_name_key in eng_text.lower(): 
            found_med_name = med_name_key
            print(f"Identified potential medicine: {found_med_name}")
            med_info_eng = get_medicine_info(found_med_name)
            break # Use the first match found

    # === Part 6: Translate Information Back to Original Language ===
    # Only translate back if the original language wasn't English
    if src_mbart_code != 'en_XX':
        # Check if the source code is actually supported by MBART as a target
        if src_mbart_code in tokenizer.lang_code_to_id:
            final_response = translate(med_info_eng, 'en_XX', src_mbart_code)
        else:
             print(f"Warning: Cannot translate back to unsupported MBART code '{src_mbart_code}'. Returning English.")
             final_response = med_info_eng # Fallback to English
    else:
        final_response = med_info_eng # Already in English

    print(f"--- Final Response: {final_response} ---")
    return final_response


In [76]:
# --- Step 6: Example Usage ---

if __name__ == "__main__":
    # === Text Input Examples ===
    print("\n=== Testing Text Input ===")
    
    # Example 1: English input
    result1 = medicine_chatbot(input_text="Tell me about Paracetamol")
    print("Bot Response (en):", result1)

    # Example 2: French input (assuming 'fr' is mapped to 'fr_XX')
    result2 = medicine_chatbot(input_text="Qu'est-ce que l'aspirine?") # "What is aspirin?"
    print("Bot Response (fr):", result2)

    # Example 2: Hindi input (assuming 'hi' is mapped to 'hi_IN')
    result3 = medicine_chatbot(input_text="इबुप्रोफेन क्या है?") # "What is Ibuprofen?"
    print("Bot Response (hi):", result3)
    
    # Example 3: Input where medicine name might not be found
    result4 = medicine_chatbot(input_text="What is good for a headache?")
    print("Bot Response (unknown med):", result4)

    # === Audio Input Example (Requires a .wav file) ===
    # Make sure you have a suitable audio file (e.g., 16kHz mono WAV recommended for SpeechBrain)
    # You might need to install ffmpeg for broader audio format support: `conda install ffmpeg` or `sudo apt install ffmpeg`
    print("\n=== Testing Audio Input (Commented Out) ===")
    audio_file = "your_audio_file.wav" # <--- !!! REPLACE WITH YOUR ACTUAL AUDIO FILE PATH !!!
    
    # try:
    #     # Ensure the file exists before calling
    #     import os
    #     if os.path.exists(audio_file) and asr_model:
    #         result_audio = medicine_chatbot(audio_path=audio_file)
    #         print(f"Bot Response (from {audio_file}):", result_audio)
    #     elif not asr_model:
    #          print(f"Skipping audio test: ASR model not loaded.")
    #     else:
    #         print(f"Skipping audio test: File '{audio_file}' not found.")
    # except Exception as e:
    #     print(f"An error occurred during the audio test: {e}")



=== Testing Text Input ===

--- Processing Text Input: Tell me about Paracetamol ---
Detected language code: en
Identified potential medicine: paracetamol
--- Final Response: Used for fever and mild pain relief. ---
Bot Response (en): Used for fever and mild pain relief.

--- Processing Text Input: Qu'est-ce que l'aspirine? ---
Detected language code: fr
Translated 'Qu'est-ce que l'aspirine?' (fr_XX) to 'What is aspirin?' (en_XX)
Identified potential medicine: aspirin
Translated 'Used to reduce pain, fever, or inflammation.' (en_XX) to 'Utilisé pour réduire la douleur, la fièvre ou l'inflammation.' (fr_XX)
--- Final Response: Utilisé pour réduire la douleur, la fièvre ou l'inflammation. ---
Bot Response (fr): Utilisé pour réduire la douleur, la fièvre ou l'inflammation.

--- Processing Text Input: इबुप्रोफेन क्या है? ---
Detected language code: hi
Translated 'इबुप्रोफेन क्या है?' (hi_IN) to 'What is ibuprofen?' (en_XX)
Identified potential medicine: ibuprofen
Translated 'Anti-inflamma